# Optimal timeplan

Ukentlig er det utallige bedrifter som setter opp vaktplaner for sine ansatte, det være seg på en restaurant, et sykehus eller store byggeprosjekt. Dette er en kjedelig og tidkrevende jobb som må tilfredstille mange krav, og som ikke nødvendigvis resulterer i den beste mulige planen. I Siemens-prosjektet ble jeg hekta på optimering og vitenskapen bak å finne løsninger på problem som beviselig er "den beste løsningen". Verktøyene man bruker til dette er grunnleggende og lavnivå, men fleksible. Som et hobbyprosjekt har jeg abstrahert disse byggeklossene til et lite rammeverk for ressursallokering som genererer optimale timeplaner.

<b><center>«Verdi – raskere – smartere»!</center></b>

## Hva er en timeplan?

En typisk timeplan ser gjerne ut som den under. Man har personer nedover som rader, og en kolonne per dag eller skift. Ved jevne perioder fyller en leder ut dette arket ved å markere en X der en gitt person skal jobbe en gitt vakt, og et tomt felt hvis personen ikke skal jobbe.

<img src="Presentations/Fagdag 2020/Timeplan eksempel.png">

## Utfordringer ved å generere en plan

Det å lage en slik timeplan kan være ganske tidkrevende. I vårt eksempel er det kun 9 ansatte som skal plasseres på 7 dager, men man kan også tenke seg arbeidssituasjoner med mange flere ansatte og enda flere skift.

<img src="Presentations/Fagdag 2020/Timeplan.png">

### Krav til skiftene
<img src="Presentations/Fagdag 2020/Timeplan - krav til skift.png">

Likevel, selv vårt relativt enkle eksempel blir fort vanskeligere når man begynner å forholde seg til enkelte krav som stilles til en god timeplan. Man har for eksempel krav til å opprettholde en minimum kapasitet på skiftene.

### Krav til de ansatte
<img src="Presentations/Fagdag 2020/Timeplan - krav til ansatte.png">

De ansatte kan også ha krav på et antall vakter som verken er for mange eller for få.

I tillegg til disse kravene, så har de ansatte ofte spesifikke og variererte ønsker og innspill til skiftene de ønsker seg, være seg spesielle skift de ønsker å jobbe på, avspassering eller medarbeidere de trives eller mistrives med. Det er mange faktorer som skal tas hensyn til.

Som man rask forstår, så kan prosessen med å sette opp en timeplan være kompleks og tidkrevende. Og enda mer deprimerrende, etter å ha brukt masse arbeid på å lage en god timeplan, så er man fremdeles ikke sikker på hvor _god_ planen egentlig er. En ting er at den tilfredstiller alle kravene, men det er mange planer som kan tilfredstille alle kravene men som likevel ikke vil anses som en god plan. For eksempel er det ikke noen vits i å plassere to team-ledere på samme skift dersom det bare trengs én, ettersom team-lederne kan være dyrere i drift enn vanlige ansatte.

<img src="Presentations/Fagdag 2020/Timeplan versus.png">

## Løsningen
Det er dette problemet rammeverket mitt forsøker å løse. Den er inspirert av paperet <a href="https://www.researchgate.net/publication/323869087_A_new_formulation_and_solution_for_the_nurse_scheduling_problem_A_case_study_in_Egypt">A new formulation and solution for the nurse scheduling problem: A case study in Egypt</a> som utledet en matematisk formulering av problemet. Rammeverket er basert på metoden Mixed-Integer Linear Programming (MILP) som er en måte å formulere optimeringsproblem på, som muliggjør å løse dem relativt rask og garanterer at løsningen er globalt optimal.

La oss ta en titt på hvordan det brukes:

In [1]:
from network.model import Network

In [2]:
# INITIALIZE PROBLEM
network = Network("ShiftScheduler")
network.setAttributes('SUPERVISOR')

### Krav til ansatte
<img src="Presentations/Fagdag 2020/Timeplan - krav til ansatte.png">

In [3]:
# WORKERS
network.addSource('Guri Malla',   cost=2500, capacityMin=2, capacityMax=2, attrs={"SUPERVISOR"})
network.addSource('Søren Klype',  cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Helle Dussen', cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Fader Ullan',  cost=1800, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Milde Moses',  cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gamle-Erik',   cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gubben Noa',   cost=1500, capacityMin=1, capacityMax=4)
network.addSource('Vittig-Per',   cost=1000, capacityMin=1, capacityMax=4)
network.addSource('Finn Urlig',   cost=800, capacityMin=1,  capacityMax=2)

### Krav til skiftene
<img src="Presentations/Fagdag 2020/Timeplan - krav til skift.png">

In [4]:
# SHIFTS
network.addSink('Monday',    capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Tuesday',   capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Wednesday', capacityMin=3, attrs={"SUPERVISOR": 1})
network.addSink('Thursday',  capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Friday',    capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Saturday',  capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Sunday',    capacityMin=3, attrs={"SUPERVISOR": 1})

In [5]:
# SOLVE
status = network.solve(optSense="minimize", objectiveMode="none", msg=False)

In [6]:
print(status)
print(f"Objective value: {network._model.objective.value()}")
if status == 'Optimal':
    result = network.getResult()
    print()
    print(result.replace(0, ' '))

Optimal
Objective value: None
Total cost: 50900.0
Happiness: 0
    > 0 requests accepted
    > 0 requests rejected

             Monday Tuesday Wednesday Thursday Friday Saturday Sunday
Guri Malla                                          1        1       
Søren Klype       1       1                  1      1                
Helle Dussen              1         1        1               1      1
Fader Ullan       1                                          1      1
Milde Moses       1                          1      1               1
Gamle-Erik        1       1         1                        1       
Gubben Noa                1         1        1      1                
Vittig-Per                                          1                
Finn Urlig                                                   1       


## Minimere utgifter
Eksempelet over tilfredstiller alle kravene, men forsøker ikke å kutte kostnadder. La oss endre det ene parameteret til solveren til `objectiveMode="cost"`.

In [7]:
# INITIALIZE PROBLEM
network = Network("ShiftScheduler")
network.setAttributes('SUPERVISOR')

# WORKERS
network.addSource('Guri Malla',   cost=2500, capacityMin=2, capacityMax=2, attrs={"SUPERVISOR"})
network.addSource('Søren Klype',  cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Helle Dussen', cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Fader Ullan',  cost=1800, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Milde Moses',  cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gamle-Erik',   cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gubben Noa',   cost=1500, capacityMin=1, capacityMax=4)
network.addSource('Vittig-Per',   cost=1000, capacityMin=1, capacityMax=4)
network.addSource('Finn Urlig',   cost=800, capacityMin=1,  capacityMax=2)

# SHIFTS
network.addSink('Monday',    capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Tuesday',   capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Wednesday', capacityMin=3, attrs={"SUPERVISOR": 1})
network.addSink('Thursday',  capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Friday',    capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Saturday',  capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Sunday',    capacityMin=3, attrs={"SUPERVISOR": 1})

# SOLVE
status = network.solve(optSense="minimize", objectiveMode="cost", msg=False)

In [8]:
print(status)
print(f"Objective value: {network._model.objective.value()}")
if status == 'Optimal':
    result = network.getResult()
    print()
    print(result.replace(0, ' '))

Optimal
Objective value: 46300.0
Total cost: 46300.0
Happiness: 0
    > 0 requests accepted
    > 0 requests rejected

             Monday Tuesday Wednesday Thursday Friday Saturday Sunday
Guri Malla                1                                  1       
Søren Klype               1         1                        1       
Helle Dussen                                 1      1               1
Fader Ullan       1       1                         1                
Milde Moses       1                                 1        1       
Gamle-Erik        1                 1        1               1       
Gubben Noa                                   1      1        1      1
Vittig-Per        1                          1      1               1
Finn Urlig                1         1                                


## Pålegge restriksjoner

### Tvinge ansatt til å jobbe eller ta fri
Programmet over fant den beste mulige løsningen på problemet. Men hva skjer hvis noen ansatte må ha fri, eller må jobbe en spesiell dag? La oss legge til to slike eksempel-restriksjoner.

In [9]:
network = Network("ShiftScheduler")
network.setAttributes('SUPERVISOR')

# WORKERS
network.addSource('Guri Malla',   cost=2500, capacityMin=2, capacityMax=2, attrs={"SUPERVISOR"})
network.addSource('Søren Klype',  cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Helle Dussen', cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Fader Ullan',  cost=1800, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Milde Moses',  cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gamle-Erik',   cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gubben Noa',   cost=1500, capacityMin=1, capacityMax=4)
network.addSource('Vittig-Per',   cost=1000, capacityMin=1, capacityMax=4)
network.addSource('Finn Urlig',   cost=800, capacityMin=1,  capacityMax=2)

# SHIFTS
network.addSink('Monday',    capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Tuesday',   capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Wednesday', capacityMin=3, attrs={"SUPERVISOR": 1})
network.addSink('Thursday',  capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Friday',    capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Saturday',  capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Sunday',    capacityMin=3, attrs={"SUPERVISOR": 1})

# CONSTRAINTS
network.addEdgeConstraint("Guri Malla",  "Monday", "enforce", "active")
network.addEdgeConstraint("Fader Ullan", "Monday", "enforce", "inactive")

# SOLVE
status = network.solve(optSense="minimize", objectiveMode="cost", msg=False)

In [10]:
print(status)
print(f"Objective value: {network._model.objective.value()}")
if status == 'Optimal':
    result = network.getResult()
    print()
    print(result.replace(0, ' '))

Optimal
Objective value: 46300.0
Total cost: 46300.0
Happiness: 0
    > 0 requests accepted
    > 0 requests rejected

             Monday Tuesday Wednesday Thursday Friday Saturday Sunday
Guri Malla        1                                 1                
Søren Klype                                  1               1      1
Helle Dussen              1         1               1                
Fader Ullan               1         1                        1       
Milde Moses                                  1               1      1
Gamle-Erik        1                                 1        1      1
Gubben Noa        1       1                  1      1                
Vittig-Per        1                          1      1        1       
Finn Urlig                1         1                                


### Lage drømme-lag og... mareritt-lag(?)
En annen avgjørelse en leder kan velge å ta hensyn til er hvilke ansatte som skal jobbe sammen, eller ikke jobbe sammen. Dette kan vi også legge til som en restriksjon

In [11]:
network = Network("ShiftScheduler")
network.setAttributes('SUPERVISOR')

# WORKERS
network.addSource('Guri Malla',   cost=2500, capacityMin=2, capacityMax=2, attrs={"SUPERVISOR"})
network.addSource('Søren Klype',  cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Helle Dussen', cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Fader Ullan',  cost=1800, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Milde Moses',  cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gamle-Erik',   cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gubben Noa',   cost=1500, capacityMin=1, capacityMax=4)
network.addSource('Vittig-Per',   cost=1000, capacityMin=1, capacityMax=4)
network.addSource('Finn Urlig',   cost=800, capacityMin=1,  capacityMax=2)

# SHIFTS
network.addSink('Monday',    capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Tuesday',   capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Wednesday', capacityMin=3, attrs={"SUPERVISOR": 1})
network.addSink('Thursday',  capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Friday',    capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Saturday',  capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Sunday',    capacityMin=3, attrs={"SUPERVISOR": 1})

# CONSTRAINTS
network.addRelationship({"Vittig-Per",   "Finn Urlig"},  mode="enforce", category="match")
network.addRelationship({"Helle Dussen", "Søren Klype"}, mode="enforce", category="mismatch")

# SOLVE
status = network.solve(optSense="minimize", objectiveMode="cost", msg=False)

In [12]:
print(status)
print(f"Objective value: {network._model.objective.value()}")
if status == 'Optimal':
    result = network.getResult()
    print()
    print(result.replace(0, ' '))

Optimal
Objective value: 47300.0
Total cost: 47300.0
Happiness: 0
    > 0 requests accepted
    > 0 requests rejected

             Monday Tuesday Wednesday Thursday Friday Saturday Sunday
Guri Malla                                          1        1       
Søren Klype       1       1                                  1       
Helle Dussen                        1        1                      1
Fader Ullan       1       1                         1                
Milde Moses                         1        1      1        1       
Gamle-Erik        1                 1        1      1        1       
Gubben Noa        1                          1      1        1       
Vittig-Per                1                                         1
Finn Urlig                1                                         1


## Fra tvang til forespørsler

### Forespørsel om vakter
Tidligere la vi til restriksjoner som dikterte at en ansatt enten måtte jobbe eller ikke fikk lov til å jobbe på en spesifikk dag. Virkeligheten er heldigvis ikke så streng, og ansatte har ofte en dialog med lederen sin hvor de kan komme med forespørsler. Det er da opp til lederen å veie disse forespørslene opp mot alle de andre kravene som må tilfredstilles. Dette kan vi også gjøre, ved å endre den ene constraint-parameteren fra `enforce` til `request`. En annen viktig endring er at vi nå må benytte en annen objektivfunksjon som kan vekte kostnadden av å ha en ansatt på en gitt vakt mot den litt mer difuse kostnadden ved å ikke innfri en forespørsel. Vi må derfor oppdatere parameterne til solveren med et nytt modus til objektivfunksjonen `objectiveMode="cost+requests"`, samt velge en kostnad for å ikke innfri en forespørsel, `pricePerRequest=250`.

In [13]:
network = Network("ShiftScheduler")
network.setAttributes('SUPERVISOR')

# WORKERS
network.addSource('Guri Malla',   cost=2500, capacityMin=2, capacityMax=2, attrs={"SUPERVISOR"})
network.addSource('Søren Klype',  cost=2300, capacityMin=2, capacityMax=3, attrs={"SUPERVISOR"})
network.addSource('Helle Dussen', cost=2300, capacityMin=2, capacityMax=3, attrs={"SUPERVISOR"})
network.addSource('Fader Ullan',  cost=1800, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Milde Moses',  cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gamle-Erik',   cost=1500, capacityMin=2, capacityMax=4)
network.addSource('Gubben Noa',   cost=1500, capacityMin=1, capacityMax=4)
network.addSource('Vittig-Per',   cost=1000, capacityMin=1, capacityMax=4)
network.addSource('Finn Urlig',   cost=800, capacityMin=1,  capacityMax=2)

# SHIFTS
network.addSink('Monday',    capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Tuesday',   capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Wednesday', capacityMin=3, attrs={"SUPERVISOR": 1})
network.addSink('Thursday',  capacityMin=4, attrs={"SUPERVISOR": 2})
network.addSink('Friday',    capacityMin=5, attrs={"SUPERVISOR": 3})
network.addSink('Saturday',  capacityMin=6, attrs={"SUPERVISOR": 3})
network.addSink('Sunday',    capacityMin=5, attrs={"SUPERVISOR": 1})

# CONSTRAINTS
network.addEdgeConstraint("Fader Ullan", "Friday",   "request", "inactive")
network.addEdgeConstraint("Fader Ullan", "Saturday", "request", "inactive")
network.addEdgeConstraint("Fader Ullan", "Sunday",   "request", "inactive")

# SOLVE
status = network.solve(optSense="minimize", objectiveMode="cost+requests", pricePerRequest=250, msg=False)

In [14]:
print(status)
print(f"Objective value: {network._model.objective.value()}")
if status == 'Optimal':
    result = network.getResult()
    print()
    print(result.replace(0, ' '))

Optimal
Objective value: 50850.0
Total cost: 50600.0
Happiness: 1
    > 2 requests accepted
    > 1 requests rejected

             Monday Tuesday Wednesday Thursday Friday Saturday Sunday
Guri Malla                                          1        1       
Søren Klype                                         1        1       
Helle Dussen                                 1      1               1
Fader Ullan       1       1         1        1               1       
Milde Moses                         1        1      1        1      1
Gamle-Erik        1       1         1                        1       
Gubben Noa        1                          1               1      1
Vittig-Per        1       1                         1               1
Finn Urlig                1                                         1


I eksempelet over innfridde vi to forespørsler. Grunnen til at den siste forespørselen ikke ble innfridd var fordi det vil kostet bedriften mer enn 250,- å innfri den. Dersom vi ønsker å prioritere forespørsler i større grad, kan vi øke denne grensen til f.eks. 1000,-

In [15]:
network = Network("ShiftScheduler")
network.setAttributes('SUPERVISOR')

# WORKERS
network.addSource('Guri Malla',   cost=2500, capacityMin=2, capacityMax=2, attrs={"SUPERVISOR"})
network.addSource('Søren Klype',  cost=2300, capacityMin=2, capacityMax=3, attrs={"SUPERVISOR"})
network.addSource('Helle Dussen', cost=2300, capacityMin=2, capacityMax=3, attrs={"SUPERVISOR"})
network.addSource('Fader Ullan',  cost=1800, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Milde Moses',  cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gamle-Erik',   cost=1500, capacityMin=2, capacityMax=4)
network.addSource('Gubben Noa',   cost=1500, capacityMin=1, capacityMax=4)
network.addSource('Vittig-Per',   cost=1000, capacityMin=1, capacityMax=4)
network.addSource('Finn Urlig',   cost=800, capacityMin=1,  capacityMax=2)

# SHIFTS
network.addSink('Monday',    capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Tuesday',   capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Wednesday', capacityMin=3, attrs={"SUPERVISOR": 1})
network.addSink('Thursday',  capacityMin=4, attrs={"SUPERVISOR": 2})
network.addSink('Friday',    capacityMin=5, attrs={"SUPERVISOR": 3})
network.addSink('Saturday',  capacityMin=6, attrs={"SUPERVISOR": 3})
network.addSink('Sunday',    capacityMin=5, attrs={"SUPERVISOR": 1})

# CONSTRAINTS
network.addEdgeConstraint("Fader Ullan", "Friday",   "request", "inactive")
network.addEdgeConstraint("Fader Ullan", "Saturday", "request", "inactive")
network.addEdgeConstraint("Fader Ullan", "Sunday",   "request", "inactive")

# SOLVE
status = network.solve(optSense="minimize", objectiveMode="cost+requests", pricePerRequest=1000, msg=False)

In [16]:
print(status)
print(f"Objective value: {network._model.objective.value()}")
if status == 'Optimal':
    result = network.getResult()
    print()
    print(result.replace(0, ' '))

Optimal
Objective value: 51100.0
Total cost: 51100.0
Happiness: 3
    > 3 requests accepted
    > 0 requests rejected

             Monday Tuesday Wednesday Thursday Friday Saturday Sunday
Guri Malla                                          1        1       
Søren Klype                                  1      1        1       
Helle Dussen                                        1        1      1
Fader Ullan       1       1         1        1                       
Milde Moses                         1        1      1        1      1
Gamle-Erik        1       1         1                        1       
Gubben Noa        1                          1               1      1
Vittig-Per        1       1                         1               1
Finn Urlig                1                                         1


### Forespørsel om arbeidskamerater
Tilsvarende kan de ansatte også komme med ønsker om medarbeider som de ønsker eller ikke ønsker å arbeide med.

In [17]:
network = Network("ShiftScheduler")
network.setAttributes('SUPERVISOR')

# WORKERS
network.addSource('Guri Malla',   cost=2500, capacityMin=2, capacityMax=2, attrs={"SUPERVISOR"})
network.addSource('Søren Klype',  cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Helle Dussen', cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Fader Ullan',  cost=1800, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Milde Moses',  cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gamle-Erik',   cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gubben Noa',   cost=1500, capacityMin=1, capacityMax=4)
network.addSource('Vittig-Per',   cost=1000, capacityMin=1, capacityMax=4)
network.addSource('Finn Urlig',   cost=800, capacityMin=1,  capacityMax=2)

# SHIFTS
network.addSink('Monday',    capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Tuesday',   capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Wednesday', capacityMin=3, attrs={"SUPERVISOR": 1})
network.addSink('Thursday',  capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Friday',    capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Saturday',  capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Sunday',    capacityMin=3, attrs={"SUPERVISOR": 1})

# CONSTRAINTS
network.addRelationship({"Milde Moses", "Gamle-Erik"}, "request", "match")

# SOLVE
status = network.solve(optSense="minimize", objectiveMode="cost+requests", pricePerRequest=250, msg=False)

In [18]:
print(status)
print(f"Objective value: {network._model.objective.value()}")
if status == 'Optimal':
    result = network.getResult()
    print()
    print(result.replace(0, ' '))

Optimal
Objective value: 45050.0
Total cost: 46300.0
Happiness: 1
    > 1 requests accepted
    > 0 requests rejected

             Monday Tuesday Wednesday Thursday Friday Saturday Sunday
Guri Malla                1                                  1       
Søren Klype       1                          1               1       
Helle Dussen                                 1      1               1
Fader Ullan               1         1               1                
Milde Moses                         1        1      1        1      1
Gamle-Erik                          1        1      1        1      1
Gubben Noa        1                                                  
Vittig-Per        1       1                         1        1       
Finn Urlig        1       1                                          


Forsøk å bytte siste parameteren i `.addRelationship()` fra `"match"` til `"mismatch"`

In [19]:
network = Network("ShiftScheduler")
network.setAttributes('SUPERVISOR')

# WORKERS
network.addSource('Guri Malla',   cost=2500, capacityMin=2, capacityMax=2, attrs={"SUPERVISOR"})
network.addSource('Søren Klype',  cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Helle Dussen', cost=2300, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Fader Ullan',  cost=1800, capacityMin=3, capacityMax=5, attrs={"SUPERVISOR"})
network.addSource('Milde Moses',  cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gamle-Erik',   cost=1500, capacityMin=2, capacityMax=5)
network.addSource('Gubben Noa',   cost=1500, capacityMin=1, capacityMax=4)
network.addSource('Vittig-Per',   cost=1000, capacityMin=1, capacityMax=4)
network.addSource('Finn Urlig',   cost=800, capacityMin=1,  capacityMax=2)

# SHIFTS
network.addSink('Monday',    capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Tuesday',   capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Wednesday', capacityMin=3, attrs={"SUPERVISOR": 1})
network.addSink('Thursday',  capacityMin=4, attrs={"SUPERVISOR": 1})
network.addSink('Friday',    capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Saturday',  capacityMin=5, attrs={"SUPERVISOR": 2})
network.addSink('Sunday',    capacityMin=3, attrs={"SUPERVISOR": 1})

# CONSTRAINTS
network.addRelationship({"Milde Moses", "Gamle-Erik"}, "request", "mismatch")

# SOLVE
status = network.solve(optSense="minimize", objectiveMode="cost+requests", pricePerRequest=250, msg=False)

In [20]:
print(status)
print(f"Objective value: {network._model.objective.value()}")
if status == 'Optimal':
    result = network.getResult()
    print()
    print(result.replace(0, ' '))

Optimal
Objective value: 46300.0
Total cost: 46300.0
Happiness: 1
    > 1 requests accepted
    > 0 requests rejected

             Monday Tuesday Wednesday Thursday Friday Saturday Sunday
Guri Malla                          1                        1       
Søren Klype       1       1                                  1       
Helle Dussen                                 1      1               1
Fader Ullan       1                                 1               1
Milde Moses                                         1               1
Gamle-Erik        1       1         1        1               1       
Gubben Noa                1                  1      1        1       
Vittig-Per        1                          1      1        1       
Finn Urlig                1         1                                


# TODO: Lag en hyggelig avslutnings-slide

# Bak kulissene til en forenklet implementasjon

## Strukturen

Tidligere har vi vist planen slik man er vant til å se den

<img src="Presentations/Fagdag 2020/Timeplan.png">

I realiteten initialiserer vi en slik tabell med symbolske variabler. Dette er variabler som i utgangspunktet ikke har en verdi. De har imidlertid informasjon om hvilke type verdi de kan ta (i vårt tilfelle er de binære, og representerer hvorvidt en gitt person jobber på et gitt skift). Når problemet er ferdig formulert, så vil vi gi det videre til en solver som vil velge de verdiene som minimerer en elelr annen objektivfunksjon. En initiel timeplan vil i vårt tilfelle da se slik ut:

<img src="Presentations/Fagdag 2020/Timeplan - under the hood table.png">

En annen representasjon av samme plan er en graf som har like mange kilde-noder som ansatte, og like mange sluk-noder som skift. Det finnes en kant mellom hver kilde- og sluk-node som kan ta verdiene 0 eller 1 utfra om den gitte personen skal jobbe på det gitte skiftet.

<img src="Presentations/Fagdag 2020/Timeplan - under the hood graph.png">

## Sette restriksjoner for en ansatt
For å sette restriksjoner for minste og strøste antall vakter en person kan jobbe, summerer man alle kantene som går ut fra en gitt kilde-node og tvinger denne til å holde seg mellom disse to verdiene.

Til illustrasjonen under ville man benyttet følgende restriksjon:
$$3 \leq \sum_{j=0}^{7}{x_{3,j}} \leq 5$$
<img src="Presentations/Fagdag 2020/Timeplan - under the hood select worker graph.png">

Dette tilsvarer å markere en rad i tabellen
<img src="Presentations/Fagdag 2020/Timeplan - under the hood select worker table.png">

## Sette restikjsoner for et skift
Tilsvarende setter man restriksjoner for minimum kapasitet til et skift ved å summere alle kantene inn til en sluk-node og tvinge denne til å være større eller lik en minimumsverdi.

Til illustrasjonen under ville man benyttet følgende restriksjon:
$$5 \leq \sum_{i=0}^{9}{x_{i,5}}$$

<img src="Presentations/Fagdag 2020/Timeplan - under the hood select shift graph.png">

Dette tilsvarer å markere en kolonne i tabellen
<img src="Presentations/Fagdag 2020/Timeplan - under the hood select shift table.png">

## Minimere kostnader

For å minimere kostnaddene, lager vi en objektivfunksjon som minimerer summen av alle ansatte multiplisert med lønnen de har, for hvert skift

$$\textrm{minimize} \sum_{i=0}^{9}{\sum_{j=0}^{7}{\textrm{kostnad}_i \bullet x_{i,j}}}$$

<img src="Presentations/Fagdag 2020/Timeplan - under the hood table.png">

## Endelig formulering
Den endelige formuleringen for et forenklet problem er som følger:

$$\textrm{minimize} \sum_{i=0}^{9}{\sum_{j=0}^{7}{\textrm{kostnad}_i \bullet x_{i,j}}}$$
$$\textrm{Subject to:}$$
$$3 \leq \sum_{j=0}^{7}{x_{3,j}} \leq 5$$
$$5 \leq \sum_{i=0}^{9}{x_{i,5}}$$

I rammeverket mitt benyttes et rammeverk som heter PuLP til å implementere objektivfunksjonen og restriksjonene. PuLP genererer deretter en såkalt LP-fil som kan leses av diverse kommersielle solvere. I vårt tilfelle benytter vi open-source solveren CBC som følger med når man installerer PuLP.